### Structured Streaming


#### Exemplo de lectura de ficheiros - Window


https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#overview

In [ ]:
# Neste exemplo, imaxina que recibes ficheiros coas consumicións dun negocio (simplificando ao límite do absurdo)
# No ficheiro indícase a data e hora (timestamp) de cada consumición e o seu tipo (café, auga, birra, ...)
# Queremos tratar esa información en "ventás temporais" para respostar a pregunta do estilo: 
# Canto café se ten consumido nos últimos 10 minutos?

In [ ]:
# No caso de de lanzar a aplicación desde consola debemos configurar a session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("StreamingFilesWindowed") \
    .getOrCreate() 

In [ ]:
from pyspark.sql.types import StructType
from pyspark.sql.functions import *

userSchema = StructType().add("hora", "timestamp").add("bebida", "string")

orixe = spark \
    .readStream \
    .option("sep", ";") \
    .schema(userSchema) \
    .csv("file:///home/hduser/input/csv")

procesado = orixe.groupBy( window(orixe.hora, "10 minutes", "5 minutes"), orixe.bebida).count()

consulta = procesado \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate",False) \
    .start()

consulta.awaitTermination()